<a href="https://colab.research.google.com/github/oneir0mancer/stable-diffusion-diffusers-colab-ui/blob/main/sd_diffusers_colab_ui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [ ]:
!pip install --upgrade diffusers accelerate transformers xformers safetensors

In [ ]:
!git clone https://huggingface.co/embed/negative /content/embeddings/negative
!git clone https://huggingface.co/embed/lora /content/Lora/positive

In [ ]:
import torch
from diffusers import DiffusionPipeline

def dummy(images, **kwargs):
    return images, False

output_index = 0

# Creating model pipeline
You can use **model_id** from [huggingface](https://huggingface.co), if the model is in diffusers format. Just check **Files** and see if there is a `model_index.json`.

Or you can download weights, and use `/content/path/to/model/dir/`.

Weights in Automatic111 format (**.ckpt** or **.safetensors** but without model_index.json) need to be converted using scripts.

In [ ]:
#@title Create pipeline
model_id_or_path = "runwayml/stable-diffusion-v1-5"  #@param {type: "string"}


pipe = DiffusionPipeline.from_pretrained(model_id_or_path).to("cuda")
pipe.safety_checker = dummy

Same rules apply for VAE. You can load it from some repository on [huggingface](https://huggingface.co) using its **model_id** and subfolder. 

Or you can load it in Automatic111 format and convert using scripts.

In [ ]:
#@title (Optional) Load VAE

from diffusers import AutoencoderKL

vae_id_or_path = "runwayml/stable-diffusion-v1-5"  #@param {type: "string"}
vae_subfolder = "vae"    #@param {type: "string"}
vae = AutoencoderKL.from_pretrained("vae_id_or_path", subfolder=vae_subfolder).to("cuda")
pipe.vae = vae

In [ ]:
#@title (Optional) Load textual inversions
#@markdown This is work in progress. Load every embedding with **.pt** extension from **embeddings** folder.

import os

for path, subdirs, files in os.walk("/content/embeddings/"):
    for name in files:
        if os.path.splitext(name)[1] != ".pt": continue
        pipe.load_textual_inversion(path, weight_name=name)
        print(path, name)

You can load any other textual inversions using 
```
pipe.load_textual_inversion("path/to/dir", weight_name="filename.pt")
```
To use them, you need to add theit token to a prompt like this `<filename>`.

Note, that it seems that if you change VAE after loading textual inversions, using them will degrade generated images.

# Generating images

In [ ]:
#@title Run this cell to generate images
prompt = "best quality, 1girl, illustration"  #@param {type: "string"}
negative_prompt = "EasyNegative, by <bad-artist-anime>, worst quality, awful quality"   #@param {type: "string"}

width = 512 #@param {type:"integer"}
height = 768    #@param {type:"integer"}
#@markdown ---

steps = 20  #@param {type: "slider", min: 1, max: 100}
CFG = 7 #@param {type: "slider", min: 1, max: 20}
seed = -1   #@param {type:"integer"}
#@markdown ---

if seed < 0: generator = torch.Generator()
else: generator = torch.manual_seed(seed)

results = pipe(prompt, negative_prompt=negative_prompt, num_inference_steps=steps, guidance_scale=CFG, 
               generator=generator, height=768, width=512)

results.images[0]

In [ ]:
#@title Save image
results.images[0].save(f"{output_index:05}.png")
output_index += 1